In [1]:
import pandas

data = pandas.read_csv("prijimaci_zkousky.csv")
data.head()

,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze
0,Květoslava,Růžičková,Elektrotechnika,62,48,51,ne,NaN
1,Jan,Moravec,Technické lyceum,81,74,66,ne,7. místo (matematická olympiáda)
2,Denis,Fiala,Informatika,63,69,65,ne,NaN
3,Luboš,Šimek,Informatika,78,79,65,ne,NaN
4,Vilém,Novotný,Informatika,84,68,78,ano,NaN


In [2]:
def evaluate_points(row):
    if row[["body_aj", "body_mat", "body_cj"]].min() < 60:
        return 0
    skore = row[["body_aj", "body_mat", "body_cj"]].sum()
    if not pandas.isnull(row["souteze"]):
        skore = skore + 10
    if row["letni_skola"] == "ano":
        skore = skore + 5
    return skore
    
    
data["body"] = data.apply(evaluate_points, axis=1)
data.to_csv("prijimaci_zkouska_body.csv")
data.head()

,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze,body
0,Květoslava,Růžičková,Elektrotechnika,62,48,51,ne,NaN,0
1,Jan,Moravec,Technické lyceum,81,74,66,ne,7. místo (matematická olympiáda),231
2,Denis,Fiala,Informatika,63,69,65,ne,NaN,197
3,Luboš,Šimek,Informatika,78,79,65,ne,NaN,222
4,Vilém,Novotný,Informatika,84,68,78,ano,NaN,235


In [3]:
data = data[data["body"] > 0]
data = data.reset_index()
data.to_csv("prijimaci_zkouska_po_vyrazeni.csv")
data.head()

,index,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze,body
0,1,Jan,Moravec,Technické lyceum,81,74,66,ne,7. místo (matematická olympiáda),231
1,2,Denis,Fiala,Informatika,63,69,65,ne,NaN,197
2,3,Luboš,Šimek,Informatika,78,79,65,ne,NaN,222
3,4,Vilém,Novotný,Informatika,84,68,78,ano,NaN,235
4,5,Drahomíra,Štěpánková,Elektrotechnika,69,74,60,ne,NaN,203


In [4]:
data["poradi"] = data.groupby("obor")["body"].rank(method="min", ascending=False)
data = data.sort_values(["obor", "poradi"])
data.to_csv("prijimaci_zkouska_poradi.csv")
data.head()

,index,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze,body,poradi
10,12,Luděk,Kratochvíl,Elektrotechnika,92,89,93,ne,NaN,274,1.0
89,143,Daniela,Poláková,Elektrotechnika,102,78,94,ne,NaN,274,1.0
8,10,Vilém,Dvořák,Elektrotechnika,98,92,80,ne,NaN,270,3.0
98,160,Barbora,Vacková,Elektrotechnika,90,83,87,ne,2. místo (přírodopisná olympiáda),270,3.0
45,72,Lubomír,Holub,Elektrotechnika,96,90,82,ne,NaN,268,5.0


In [5]:
def prijat(row):
    if row["poradi"] <= 30:
        return "Ano"
    else:
        return "Ne"

data["prijat"] = data.apply(prijat, axis=1)
data.to_csv("prijimaci_zkouska_rozhodnuti.csv")
data

,index,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze,body,poradi,prijat
10,12,Luděk,Kratochvíl,Elektrotechnika,92,89,93,ne,NaN,274,1.0,Ano
89,143,Daniela,Poláková,Elektrotechnika,102,78,94,ne,NaN,274,1.0,Ano
8,10,Vilém,Dvořák,Elektrotechnika,98,92,80,ne,NaN,270,3.0,Ano
98,160,Barbora,Vacková,Elektrotechnika,90,83,87,ne,2. místo (přírodopisná olympiáda),270,3.0,Ano
45,72,Lubomír,Holub,Elektrotechnika,96,90,82,ne,NaN,268,5.0,Ano
...,...,...,...,...,...,...,...,...,...,...,...,...
54,87,Veronika,Poláková,Technické lyceum,68,68,61,ano,NaN,202,27.0,Ano
25,38,Monika,Kratochvílová,Technické lyceum,60,76,65,ne,NaN,201,28.0,Ano
11,15,Erik,Dvořák,Technické lyceum,68,71,61,ne,NaN,200,29.0,Ano
119,195,Naděžda,Čermáková,Technické lyceum,71,61,67,ne,NaN,199,30.0,Ano


Řešení z videa včetně bonusů je níže.

In [1]:
import pandas

def get_points(row):
    # Pokud dostane z některého z testů méně než 60 bodů, dostane hodnocení 0.
    if row["body_aj":"body_cj"].min() < 60:
        points = 0
    else:
        points = row["body_aj":"body_cj"].sum()
        # pokud hodnota ve sloupečku souteze není prázdná
        if not pandas.isnull(row["souteze"]):
            # Pokud se uchazeč hlásí na obor Elektrotechnika nebo Informatika 
            if row["obor"] in ["Elektrotechnika", "Informatika"]:
                # přiděluj bonusové body pouze za fyzikální nebo matematickou olympiádu
                if "matematická" in row["souteze"] or "fyzikální" in row["souteze"]:
                    # Za 1. až 3. místo v olympiádě přiděluj 15 bodů, za nižší místa 5.
                    if row["souteze"][0] in ["1", "2", "3"] and row["souteze"][1] == ".":
                        points = points + 15
                    else:
                        points = points + 5
            else:
                if row["souteze"][0] in ["1", "2", "3"] and row["souteze"][1] == ".":
                    points = points + 15
                else:
                    points = points + 5
        # pokud hodnota ve sloupečku letni_skola je ano
        if row["letni_skola"] == "ano":
            points = points + 5
    return points

def decision(row):
    if row["poradi"] <= 30:
        decision = "ano"
    else:
        decision = "ne"
    return decision


data = pandas.read_csv("prijimaci_zkousky.csv")
data["body"] = data.apply(get_points, axis=1)
#  Dále vytvoř novou tabulku, která obsahuje pouze uchazeče a uchazečky, kteří mají nenulové hodnocení.
data = data[data["body"] > 0]
# Dále využij metodu rank() k výpočtu pořadí uchazeče/uchazečky v rámci daného oboru,
data["poradi"] = data.groupby("obor")["body"].rank(ascending=False, method="min")
# Data seřaď podle oboru a počtu bodů
data = data.sort_values(["obor", "poradi"])
# Nakonec vytvoř tabulku s přijatými uchazeči, kde budou uchazeči hodnocení od 1. do 30. místa pro daný obor.
data["prijat"] = data.apply(decision, axis=1)
data_prijati = data[data["prijat"] == "ano"]
print(data_prijati.tail(50))


      prijmeni  krestni_jmeno              obor  body_aj  body_mat  body_cj   
106      Radek          Urban       Informatika       76        83       79  \
109       Erik        Kopecký       Informatika       83        73       81   
122       René         Veselý       Informatika       79        76       81   
4        Vilém        Novotný       Informatika       84        68       78   
112   Kateřina       Horáková       Informatika       80        76       74   
84   Miroslava      Soukupová       Informatika       72        74       83   
199     Miluše        Novotná       Informatika       67        79       75   
76       Lukáš         Soukup       Informatika       76        74       75   
92   Alexandra        Šimková       Informatika       72        80       68   
3        Luboš          Šimek       Informatika       78        79       65   
40     Markéta  Kratochvílová       Informatika       79        75       68   
77        Erik          Zeman       Informatika     